## Group member: Hongyuan Zhang

In [393]:
from bs4 import BeautifulSoup
import bs4 as bs
from bs4 import SoupStrainer
import urllib
import urllib.request, urllib.error, urllib.parse
import pandas as pd
import csv
import codecs
import datetime

In [394]:
df = pd.read_csv('HW_Mutual_Fund_Info_Table_Link.csv')
df.head()

,CIK,Name,Date,Information Table Links,File Type,
0,1311981,Adirondack_Funds,2015-11-10,/Archives/edgar/data/1311981/00011620441500116...,xml,NaN
1,1311981,Adirondack_Funds,2013-05-06,/Archives/edgar/data/1311981/00011620441300051...,text,NaN
2,813470,ADVANCE_CAPITAL_I_INC,2016-11-01,/Archives/edgar/data/813470/000151597116000610...,xml,NaN
3,813470,ADVANCE_CAPITAL_I_INC,2016-08-09,/Archives/edgar/data/813470/000151597116000570...,xml,NaN
4,813470,ADVANCE_CAPITAL_I_INC,2016-05-09,/Archives/edgar/data/813470/000151597116000494...,xml,NaN


In [395]:
#filter data, separate them to xlm and txt

print(list(df.columns.values))
df[' Date'] = pd.to_datetime(df[' Date'], format="%Y-%m-%d", errors='ignore')
txt_file = df[df[' File Type'] == 'text']
xml_file = df[df[' File Type'] == 'xml']
txt_file = txt_file[txt_file[' Date'] > '2010-12-31']
xml_file = xml_file[xml_file[' Date'] > '2010-12-31']
txt_file = txt_file[[' Name', ' Date', ' Information Table Links']]
xml_file = xml_file[[' Name', ' Date', ' Information Table Links']]
txt_file

['CIK', ' Name', ' Date', ' Information Table Links', ' File Type', ' ']


,Name,Date,Information Table Links
1,Adirondack_Funds,2013-05-06,/Archives/edgar/data/1311981/00011620441300051...
16,ADVANCE_CAPITAL_I_INC,2013-05-01,/Archives/edgar/data/813470/000081347013000006...
17,ADVANCE_CAPITAL_I_INC,2013-02-08,/Archives/edgar/data/813470/000081347013000001...
18,ADVANCE_CAPITAL_I_INC,2012-11-07,/Archives/edgar/data/813470/000081347012000023...
19,ADVANCE_CAPITAL_I_INC,2012-08-09,/Archives/edgar/data/813470/000081347012000019...
20,ADVANCE_CAPITAL_I_INC,2012-05-15,/Archives/edgar/data/813470/000081347012000014...
21,ADVANCE_CAPITAL_I_INC,2012-02-02,/Archives/edgar/data/813470/000081347012000003...
104,Global_X_Funds,2011-02-14,/Archives/edgar/data/1432353/00011442041100842...


### For txt file

In [396]:
import requests
import random 
from collections import OrderedDict

headers_list = [
    # Firefox 77 Mac
    {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Firefox 77 Windows
    {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 83 Mac
    {
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
    # Chrome 83 Windows 
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    }
]

In [397]:
type(xml_file[' Name'].tolist())

list

In [398]:
issue_date = []
mutual_fund_name = [] 
name_of_issuer = [] 
CUSIP = [] # CUSIP number
shares = [] # No. of Shares of the company in the Mutual Fund

# Your code goes here
repeat = []
number = 0

for url in txt_file[' Information Table Links'].tolist():
    print(url)
    link = "https://www.sec.gov" + url
    print(link)  

    headers = random.choice(headers_list)
    r = requests.Session()
    r.headers = headers

    html = r.get(link).text

    html = html.lower()

    start = html.find('name of issuer')
    end = html.find('</document>')
    html2 = html[start:end]
    html2 = html2.replace(" adr ", " adr  ")
    html2 = html2.replace(" $0.01 ", " $0.01   ")
    html2 = html2.replace(" $.001 ", " $0.01   ")
    html2 = html2.replace(" com com ", "  com com ")
    html2 = html2.replace(" $ ", "  ")
    html2 = html2.replace(" 892 ", " 892  ")
    html2 = html2.replace(" sole ", "  sole ")
    html2 = html2.replace("<s>", " ")
    html2 = html2.replace("<c>", " ")
    html2 = html2.replace("-", " ")
    html2 = html2.replace(" ma ", " ")
    html2 = html2.replace(" new ", " ")
    html2 = html2.replace(" cl ", " ")
    html2 = html2.replace("cl b ", " ")
    html2 = html2.replace(" rep ", " ")
    html2 = html2.replace(" pfd ", " ")
    html2 = html2.replace(" n v ", " ")
    html2 = html2.replace(" sek ", " ")
    html2 = html2.replace(" ser ", " ")
    html2 = html2.replace(" pref ", " ")

    start = html2.find("none\n")+5
    html2 = html2[start:]

    #print(html2.splitlines())

    html2 = html2.splitlines()

    for i in range(0, len(html2)):
        entrylist = html2[i].split("  ")
        try:
            while True:
                entrylist.remove('')
        except ValueError:
            pass

        if len(entrylist) > 4 and entrylist[0] != "name of issuer":
            name_of_issuer.append(entrylist[0])
            CUSIP.append(entrylist[2])
            shares.append(entrylist[4])  
            issue_date.append(txt_file[' Date'].tolist()[number])
            mutual_fund_name.append(txt_file[' Name'].tolist()[number])
            
#     print(len(name_of_issuer))
#     print(len(CUSIP))
#     print(len(shares))
#     print(len(issue_date))
#     print(len(mutual_fund_name))
    number += 1

/Archives/edgar/data/1311981/000116204413000513/0001162044-13-000513.txt
https://www.sec.gov/Archives/edgar/data/1311981/000116204413000513/0001162044-13-000513.txt
/Archives/edgar/data/813470/000081347013000006/0000813470-13-000006.txt
https://www.sec.gov/Archives/edgar/data/813470/000081347013000006/0000813470-13-000006.txt
/Archives/edgar/data/813470/000081347013000001/0000813470-13-000001.txt
https://www.sec.gov/Archives/edgar/data/813470/000081347013000001/0000813470-13-000001.txt
/Archives/edgar/data/813470/000081347012000023/0000813470-12-000023.txt
https://www.sec.gov/Archives/edgar/data/813470/000081347012000023/0000813470-12-000023.txt
/Archives/edgar/data/813470/000081347012000019/0000813470-12-000019.txt
https://www.sec.gov/Archives/edgar/data/813470/000081347012000019/0000813470-12-000019.txt
/Archives/edgar/data/813470/000081347012000014/0000813470-12-000014.txt
https://www.sec.gov/Archives/edgar/data/813470/000081347012000014/0000813470-12-000014.txt
/Archives/edgar/data

In [399]:
issue_date = [date_obj.strftime('%Y-%m-%d') for date_obj in issue_date]
info_date[1:10]

['2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10']

In [400]:
#making csv
with open('hw3_txt.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['issue_date', 'mutual_fund_name', 'name_of_issuer', 'CUSIP', 'shares'])
    writer.writerows(zip(issue_date, mutual_fund_name, name_of_issuer, CUSIP, shares))

## for xml file

In [401]:
mutual_fund_name = []
info_date = []
name_of_issuer = []
title_of_class = []
CUSIP = []
Value = []
Shares = []
SHRS_or_PRN = []
SH_PRN = []
PUTorCALL = []
Inv_Discretion = []
Other_Manager = []
Authority_Sole = []
Authority_Shared = []
Authority_None = []

In [402]:
xml_date = xml_file[' Date'].tolist()
xml_Name = xml_file[' Name'].tolist()
xml_link = xml_file[' Information Table Links'].tolist()
xml_link = ["https://www.sec.gov"+i for i in xml_link]
# print(xml_date)
# print(xml_Name)
print(xml_link)

['https://www.sec.gov/Archives/edgar/data/1311981/000116204415001166/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/813470/000151597116000610/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/813470/000151597116000570/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/813470/000151597116000494/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/813470/000151597116000435/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/813470/000151597115000375/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/813470/000151597115000255/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/813470/000151597115000139/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/813470/000151597115000031/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/813470/000151597114000479/xslForm13F_X01/infotable.xml', 'https://www.sec.g

In [403]:
import time

for link_number in range(0,len(xml_link)):
    headers = random.choice(headers_list)
    r = requests.Session()
    r.headers = headers
    html = r.get(xml_link[link_number]).text
    soup = bs.BeautifulSoup(html)
    infotable = soup.find_all("table", summary = "Form 13F-NT Header Information")
    
    for row in infotable[0].find_all('tr')[3:]:
        print(row)
        columns = row.find_all('td')
        mutual_fund_name.append(xml_Name[link_number])
        info_date.append(xml_date[link_number])
        name_of_issuer.append(columns[0].getText())
        title_of_class.append(columns[1].getText())
        CUSIP.append(columns[2].getText())
        Value.append(columns[3].getText())
        SHRS_or_PRN.append(columns[4].getText())
        SH_PRN.append(columns[5].getText())
        PUTorCALL.append(columns[6].getText())
        Inv_Discretion.append(columns[7].getText())
        Other_Manager.append(columns[8].getText())
        Authority_Sole.append(columns[9].getText())
        Authority_Shared.append(columns[10].getText())
        Authority_None.append(columns[11].getText())
        
#     time.sleep(random.uniform(5,15))

<tr>
<td class="FormData">ABBOTT LABS</td><td class="FormData">COM</td><td class="FormData">2824100</td><td class="FormDataR">266</td><td class="FormDataR">6,625</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">0</td><td class="FormDataR">0</td><td class="FormDataR">6,625</td>
</tr>
<tr>
<td class="FormData">ABBVIE INC</td><td class="FormData">COM</td><td class="FormData">00287Y109</td><td class="FormDataR">343</td><td class="FormDataR">6,300</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">0</td><td class="FormDataR">0</td><td class="FormDataR">6,300</td>
</tr>
<tr>
<td class="FormData">ALCOA INC</td><td class="FormData">COM</td><td class="FormData">13817101</td><td class="FormDataR">442</td><td class="FormDataR">45,800</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">0</td><td class="FormDataR">0</td><td class="FormDataR"

<tr>
<td class="FormData">AARONS INC</td><td class="FormData">COMMON STOCK</td><td class="FormData">002535300</td><td class="FormDataR">464</td><td class="FormDataR">21,200</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">21,200</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">ADVANCE AUTO PARTS</td><td class="FormData">COMMON STOCK</td><td class="FormData">00751Y106</td><td class="FormDataR">1,212</td><td class="FormDataR">7,500</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">7,500</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">APPLE COMPUTER</td><td class="FormData">COMMON STOCK</td><td class="FormData">037833100</td><td class="FormDataR">564</td><td class="FormDataR">5,900</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">5,900</td>

<tr>
<td class="FormData">3M CO.</td><td class="FormData">COMMON STOCK</td><td class="FormData">88579Y101</td><td class="FormDataR">1,115</td><td class="FormDataR">7,400</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">7,400</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">ACUITY BRANDS INC</td><td class="FormData">COMMON STOCK</td><td class="FormData">00508Y102</td><td class="FormDataR">1,075</td><td class="FormDataR">4,600</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">4,600</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">ADT CORP/THE</td><td class="FormData">COMMON STOCK</td><td class="FormData">00101J106</td><td class="FormDataR">204</td><td class="FormDataR">6,200</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">6,200</td><td cla

<tr>
<td class="FormData">3M CO.</td><td class="FormData">COMMON STOCK</td><td class="FormData">88579Y101</td><td class="FormDataR">586</td><td class="FormDataR">3,800</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">3,800</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">ACUITY BRANDS INC</td><td class="FormData">COMMON STOCK</td><td class="FormData">00508Y102</td><td class="FormDataR">756</td><td class="FormDataR">4,200</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">4,200</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">ADVANCE AUTO PARTS</td><td class="FormData">COMMON STOCK</td><td class="FormData">00751Y106</td><td class="FormDataR">1,481</td><td class="FormDataR">9,300</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">9,300</td><td

</tr>
<tr>
<td class="FormData">INTERPUBLIC GROUP COMPANIES</td><td class="FormData">COMMON STOCK</td><td class="FormData">460690100</td><td class="FormDataR">703</td><td class="FormDataR">31,800</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">31,800</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">INVESCO LTD</td><td class="FormData">COMMON STOCK</td><td class="FormData">G491BT108</td><td class="FormDataR">1,961</td><td class="FormDataR">49,400</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">49,400</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">ISHARESMIDCAP 400</td><td class="FormData">COMMON STOCK</td><td class="FormData">464287507</td><td class="FormDataR">1,641</td><td class="FormDataR">10,800</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td clas

<tr>
<td class="FormData">APPLE COMPUTER</td><td class="FormData">COMMON STOCK</td><td class="FormData">037833100</td><td class="FormDataR">1,874</td><td class="FormDataR">18,600</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">18,600</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">ACE Limited</td><td class="FormData">COMMON STOCK</td><td class="FormData">H0023R105</td><td class="FormDataR">1,720</td><td class="FormDataR">16,400</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">16,400</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">AFLAC, INC.</td><td class="FormData">COMMON STOCK</td><td class="FormData">001055102</td><td class="FormDataR">1,579</td><td class="FormDataR">27,100</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">27,100</t

<tr>
<td class="FormData">AMERISOURCEBERGEN CORP</td><td class="FormData">COMMON STOCK</td><td class="FormData">03073E105</td><td class="FormDataR">1,200</td><td class="FormDataR">18,300</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">18,300</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">AMERICAN CAMPUS CMNTYS INC</td><td class="FormData">COMMON STOCK</td><td class="FormData">024835100</td><td class="FormDataR">1,236</td><td class="FormDataR">33,100</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">33,100</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">ACE LTD</td><td class="FormData">COMMON STOCK</td><td class="FormData">H0023R105</td><td class="FormDataR">1,258</td><td class="FormDataR">12,700</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="F

</tr>
<tr>
<td class="FormData">CONCUR TECHNOLOGIES</td><td class="FormData">COMMON STOCK</td><td class="FormData">206708109</td><td class="FormDataR">681</td><td class="FormDataR">6,600</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">6,600</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">CON-WAY INC.</td><td class="FormData">COMMON STOCK</td><td class="FormData">205944101</td><td class="FormDataR">608</td><td class="FormDataR">15,300</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">15,300</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">CAPITAL ONE FINANCIAL CORP</td><td class="FormData">COMMON STOCK</td><td class="FormData">14040H105</td><td class="FormDataR">1,417</td><td class="FormDataR">18,500</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class=

</tr>
<tr>
<td class="FormData">HEALTH NET INC</td><td class="FormData">COMMON STOCK</td><td class="FormData">42222G108</td><td class="FormDataR">231</td><td class="FormDataR">7,300</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">7,300</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">HOLOGIC INC</td><td class="FormData">COMMON STOCK</td><td class="FormData">436440101</td><td class="FormDataR">510</td><td class="FormDataR">24,700</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">24,700</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">HELMERICH &amp; PAYNE</td><td class="FormData">COMMON STOCK</td><td class="FormData">423452101</td><td class="FormDataR">400</td><td class="FormDataR">5,800</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">5,

</tr>
<tr>
<td class="FormData">ASSOCIATED BANC-CORP</td><td class="FormData">COMMON STOCK</td><td class="FormData">045487105</td><td class="FormDataR">257</td><td class="FormDataR">16,500</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">16,500</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">ASHLAND INC.</td><td class="FormData">COMMON STOCK</td><td class="FormData">044209104</td><td class="FormDataR">593</td><td class="FormDataR">7,100</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormDataR">7,100</td><td class="FormDataR">0</td><td class="FormDataR">0</td>
</tr>
<tr>
<td class="FormData">ASCENA RETAIL GROUP</td><td class="FormData">COMMON STOCK</td><td class="FormData">04351G101</td><td class="FormDataR">1,049</td><td class="FormDataR">60,100</td><td class="FormData">SH</td><td> </td><td class="FormData">SOLE</td><td> </td><td class="FormD

In [404]:
info_date = [date_obj.strftime('%Y-%m-%d') for date_obj in info_date]
info_date[1:10]

['2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10']

In [405]:
len(PUTorCALL)

2830

In [406]:
info_date[1:10]

['2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10',
 '2015-11-10']

In [407]:
len(info_date)

2830

In [408]:
len(name_of_issuer)

2830

In [409]:
# making csv
with open('hw3_xml.csv', 'w+', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['issue_date','mutual_fund_name', 'name_of_issuer', 'CUSIP', 'shares'])
    writer.writerows(zip(info_date, mutual_fund_name, name_of_issuer, CUSIP, SHRS_or_PRN))

## combine both files


In [410]:
txt = pd.read_csv('hw3_txt.csv')
xml = pd.read_csv('hw3_xml.csv')
xml.head()
print(len(txt))
print(len(xml))

2268
2830


In [411]:
print(txt.dtypes)
print(xml.dtypes)

issue_date          object
mutual_fund_name    object
name_of_issuer      object
CUSIP               object
shares              object
dtype: object
issue_date          object
mutual_fund_name    object
name_of_issuer      object
CUSIP               object
shares              object
dtype: object


In [412]:
combine = pd.concat([txt, xml])
print(len(combine))
combine.dtypes

5098


issue_date          object
mutual_fund_name    object
name_of_issuer      object
CUSIP               object
shares              object
dtype: object

In [413]:
combine.to_csv('hw3_output.csv', index=False)